# load packages

In [1]:
fig_save_path = '/data/peer/wallet/Jupyter/2023_Particle_SASP/figures/'
DATA_DIR = '/data/peer/wallet/Jupyter/2023_Particle_SASP/data/'

In [2]:
# import packages 
import scanpy as sc
import scanpy.external as sce
import anndata as anndata
from anndata import AnnData
import time
import os
import numpy as np
import seaborn as sns
import pandas as pd
import re
import h5py
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as patches
import matplotlib.colors as colors
import matplotlib.font_manager

# other packages
import glob
from collections import OrderedDict
from scipy.sparse import csr_matrix
#import diffxpy.api as de
from scanpy.external.pp import magic
from scipy.stats import zscore
import scipy.cluster.hierarchy as hc
from statistics import median
import math
from typing import Union
import phenograph
import numpy.matlib

from kneed import KneeLocator


In [3]:
#custom libraries
from scrnaseq_preprocessing_utils import *

In [4]:
#set style
sns.set_style('white')
# define colors
color_map_clusters_nogrey = ["#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
"#5A0007", "#809693", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
"#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
"#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
"#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
"#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
"#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
"#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
"#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
"#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
"#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
"#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C"]

# annotate cells

In [49]:

adata_path = DATA_DIR + 'TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo.h5ad'
adata_save_path = DATA_DIR + 'TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_annotated.h5ad'
csv_save_path = DATA_DIR + 'TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-annotations.csv'
#annotations
adata_B_path = '/data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_immune_2500hv_annotated_BP_2500hv.h5ad'
adata_M_path = '/data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_immune_2500hv_annotated_M_5000hv.h5ad'
adata_TNK_path = DATA_DIR + 'TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_immune_2500hv_annotated_TNK_2500hv.h5ad'
adata_ILC_path = '/data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_immune_2500hv_annotated_TNK_2500hv_ILC_2500.h5ad'

adata_paths = [adata_B_path, adata_M_path, adata_TNK_path,adata_ILC_path] 

make_raw= False


In [6]:
#load adata
adata = sc.read(adata_path)
adata_annotations = []

for i in adata_paths:
    adata_subset = sc.read(i)
    adata_annotations.append(adata_subset)
    del adata_subset

adata.obs_names #check whether format is barcode_sampleid

Index(['TTTCATGCATGACTGT-1-1', 'ATGAGTCAGGGTACAC-1-1', 'TCCTTTCAGAGATGCC-1-1',
       'GCAGCCATCCACTTCG-1-1', 'TACGTCCGTCGCTTAA-1-1', 'TGATTTCAGCGCCTTG-1-1',
       'ATGCGATTCGTAGGGA-1-1', 'GTCGTAATCGTGCATA-1-1', 'TATTGGGAGACGCCCT-1-1',
       'CACGTGGAGTTGGCGA-1-1',
       ...
       'TGTCCCATCAAGTGGG-1-45', 'ATCCACCGTTAAGTCC-1-45',
       'TATTGCTGTAACATGA-1-45', 'GTCGTAAGTGTATACC-1-45',
       'CCCTTAGTCAGTGATC-1-45', 'CTGTAGAAGATCGACG-1-45',
       'GTGGAAGTCAGAGTTC-1-45', 'CTAACCCAGCTCGAAG-1-45',
       'CTCCTTTCACAATGAA-1-45', 'TCGTGCTAGCGATGCA-1-45'],
      dtype='object', length=207661)

In [7]:
print(adata.raw.X)

  (0, 22140)	1.9689747
  (0, 22139)	0.70666605
  (0, 22138)	1.4065048
  (0, 22137)	2.102981
  (0, 22136)	1.1165972
  (0, 22135)	2.102981
  (0, 22134)	3.1166945
  (0, 22133)	2.102981
  (0, 22131)	3.0212638
  (0, 22130)	2.102981
  (0, 22129)	2.4223406
  (0, 22128)	2.2211332
  (0, 22125)	0.70666605
  (0, 22121)	0.70666605
  (0, 22120)	1.1165972
  (0, 22119)	1.1165972
  (0, 22115)	0.70666605
  (0, 22092)	1.1165972
  (0, 22091)	0.70666605
  (0, 22058)	0.70666605
  (0, 22057)	1.1165972
  (0, 22054)	1.1165972
  (0, 22042)	0.70666605
  (0, 22025)	0.70666605
  (0, 22023)	0.70666605
  :	:
  (207660, 4575)	4.2686133
  (207660, 4490)	4.2686133
  (207660, 4343)	4.2686133
  (207660, 4288)	4.2686133
  (207660, 4264)	4.2686133
  (207660, 3953)	4.2686133
  (207660, 3890)	4.2686133
  (207660, 3460)	4.2686133
  (207660, 3398)	4.2686133
  (207660, 3333)	4.2686133
  (207660, 3236)	4.2686133
  (207660, 3220)	4.2686133
  (207660, 3203)	4.2686133
  (207660, 3194)	4.2686133
  (207660, 3013)	4.9547353
  (207660

In [18]:
#retrieve annotations from .obs element and convert into dict

annotation_subtypes_2 = []
annotation_subtypes_3 = []

for i in adata_annotations:
    annotation_subtypes_2.append(i.obs['annotation'])
    annotation_subtypes_3.append(i.obs['pheno_annotate'])
    
annotation_subtype_dicts_1 = []


annotation_subtype_dicts_2 = []
    
for i in annotation_subtypes_2:
    annotation_subtype_2_dict = i.to_dict()
    annotation_subtype_dicts_2.append(annotation_subtype_2_dict)
    
    
annotation_subtype_dicts_3 = []
    
for i in annotation_subtypes_3:
    annotation_subtype_3_dict = i.to_dict()
    annotation_subtype_dicts_3.append(annotation_subtype_3_dict)
    


In [19]:
#concatenate dictionaries order matters! sutype 1
    
annotation_subtype_dict_2 = {}

for i in annotation_subtype_dicts_2:
    annotation_subtype_dict_2.update(i)
        
annotation_subtype_dict_3 = {}

for i in annotation_subtype_dicts_3:
    annotation_subtype_dict_3.update(i)   


In [20]:
adata.obs['annotation'] = adata.obs_names.map(annotation_subtype_dict_2)
adata.obs['pheno_annotate'] = adata.obs_names.map(annotation_subtype_dict_3)

In [21]:
set(adata.obs['annotation'])

{'B',
 'B-T_doublet',
 'B-memory-DN',
 'B-naive',
 'B_proliferating',
 'CD4-T-naive',
 'CD4-TSCM_TFH',
 'CD56bright-NK',
 'CD56dim-NK',
 'CD8-T',
 'CD8-TRM',
 'CD8-T_KLRG1pos-effector',
 'GC-B',
 'Mac',
 'NK',
 'PV-Mac',
 'T',
 'T_proliferating',
 'Treg',
 'c-mono',
 'cDC1',
 'cDC2',
 'cDC3',
 'gran',
 'mast',
 nan,
 'nc-mono',
 'p-DC',
 'plasma'}

In [22]:
set(adata.obs['pheno_annotate'])

{'BP_0',
 'BP_1',
 'BP_10',
 'BP_2',
 'BP_3',
 'BP_4',
 'BP_5',
 'BP_6',
 'BP_7',
 'BP_8',
 'BP_9',
 'ILC_0',
 'ILC_1',
 'ILC_2',
 'ILC_3',
 'ILC_4',
 'M_0',
 'M_1',
 'M_10',
 'M_11',
 'M_12',
 'M_13',
 'M_14',
 'M_15',
 'M_16',
 'M_17',
 'M_18',
 'M_19',
 'M_2',
 'M_20',
 'M_21',
 'M_22',
 'M_23',
 'M_3',
 'M_4',
 'M_5',
 'M_6',
 'M_7',
 'M_8',
 'M_9',
 'TNK_0',
 'TNK_1',
 'TNK_11',
 'TNK_12',
 'TNK_13',
 'TNK_14',
 'TNK_15',
 'TNK_16',
 'TNK_2',
 'TNK_3',
 'TNK_4',
 'TNK_5',
 'TNK_6',
 'TNK_7',
 'TNK_8',
 'TNK_pheno_var_TNK_k10_16',
 nan}

In [25]:
adata.obs['annotation'].isna().value_counts()

annotation
True     187553
False     20108
Name: count, dtype: int64

In [28]:
adata[adata.obs['annotation'].isna()].obs['Level 3 Annotation']

TTTCATGCATGACTGT-1-1     Malignant
ATGAGTCAGGGTACAC-1-1      Pericyte
TCCTTTCAGAGATGCC-1-1     Malignant
GCAGCCATCCACTTCG-1-1      Pericyte
TACGTCCGTCGCTTAA-1-1     Malignant
                           ...    
CTGTAGAAGATCGACG-1-45       Ductal
GTGGAAGTCAGAGTTC-1-45       Ductal
CTAACCCAGCTCGAAG-1-45       Ductal
CTCCTTTCACAATGAA-1-45       Ductal
TCGTGCTAGCGATGCA-1-45       Ductal
Name: Level 3 Annotation, Length: 187553, dtype: category
Categories (21, object): ['ADM', 'Acinar', 'Acinar (REG+)', 'Adipocyte', ..., 'Schwann', 'Vascular', 'Vascular smooth muscle', 'myCAF']

In [26]:
adata.obs['pheno_annotate'].isna().value_counts()

pheno_annotate
True     187553
False     20108
Name: count, dtype: int64

In [30]:
set(adata.obs['Level 1 Annotation'])

{'Adipocyte',
 'Cancer-associated fibroblast',
 'Endocrine',
 'Endothelial',
 'Epithelial (malignant)',
 'Epithelial (non-malignant)',
 'Intra-pancreatic neurons',
 'Lymphoid',
 'Myeloid',
 'Pericyte',
 'Schwann',
 'Vascular smooth muscle'}

## add non-immune annotaitons

In [32]:
new_pheno = []
new_annotations = []

old_pheno = list(adata.obs['pheno_annotate'])
old_annotations = list(adata.obs['annotation'])
author_annotations = list(adata.obs['Level 3 Annotation'])

for i,v in enumerate(list(adata.obs['Level 1 Annotation'])):
    if v in ['Lymphoid','Myeloid']:
        new_pheno.append(old_pheno[i])
        new_annotations.append(old_annotations[i])
    else:
        new_pheno.append('all_'+author_annotations[i])
        new_annotations.append('all_'+author_annotations[i])

In [33]:
set(new_pheno)

{'BP_0',
 'BP_1',
 'BP_10',
 'BP_2',
 'BP_3',
 'BP_4',
 'BP_5',
 'BP_6',
 'BP_7',
 'BP_8',
 'BP_9',
 'ILC_0',
 'ILC_1',
 'ILC_2',
 'ILC_3',
 'ILC_4',
 'M_0',
 'M_1',
 'M_10',
 'M_11',
 'M_12',
 'M_13',
 'M_14',
 'M_15',
 'M_16',
 'M_17',
 'M_18',
 'M_19',
 'M_2',
 'M_20',
 'M_21',
 'M_22',
 'M_23',
 'M_3',
 'M_4',
 'M_5',
 'M_6',
 'M_7',
 'M_8',
 'M_9',
 'TNK_0',
 'TNK_1',
 'TNK_11',
 'TNK_12',
 'TNK_13',
 'TNK_14',
 'TNK_15',
 'TNK_16',
 'TNK_2',
 'TNK_3',
 'TNK_4',
 'TNK_5',
 'TNK_6',
 'TNK_7',
 'TNK_8',
 'TNK_pheno_var_TNK_k10_16',
 'all_ADM',
 'all_Acinar',
 'all_Acinar (REG+)',
 'all_Adipocyte',
 'all_Alpha',
 'all_Beta',
 'all_CAF',
 'all_Delta',
 'all_Ductal',
 'all_Ductal (atypical)',
 'all_Epsilon',
 'all_Gamma',
 'all_Hormone-negative neuroendocrine',
 'all_Intra-pancreatic neurons',
 'all_Lymphatic',
 'all_Malignant',
 'all_Pericyte',
 'all_Schwann',
 'all_Vascular',
 'all_Vascular smooth muscle',
 'all_myCAF'}

In [34]:
set(new_annotations)

{'B',
 'B-T_doublet',
 'B-memory-DN',
 'B-naive',
 'B_proliferating',
 'CD4-T-naive',
 'CD4-TSCM_TFH',
 'CD56bright-NK',
 'CD56dim-NK',
 'CD8-T',
 'CD8-TRM',
 'CD8-T_KLRG1pos-effector',
 'GC-B',
 'Mac',
 'NK',
 'PV-Mac',
 'T',
 'T_proliferating',
 'Treg',
 'all_ADM',
 'all_Acinar',
 'all_Acinar (REG+)',
 'all_Adipocyte',
 'all_Alpha',
 'all_Beta',
 'all_CAF',
 'all_Delta',
 'all_Ductal',
 'all_Ductal (atypical)',
 'all_Epsilon',
 'all_Gamma',
 'all_Hormone-negative neuroendocrine',
 'all_Intra-pancreatic neurons',
 'all_Lymphatic',
 'all_Malignant',
 'all_Pericyte',
 'all_Schwann',
 'all_Vascular',
 'all_Vascular smooth muscle',
 'all_myCAF',
 'c-mono',
 'cDC1',
 'cDC2',
 'cDC3',
 'gran',
 'mast',
 'nc-mono',
 'p-DC',
 'plasma'}

In [35]:
adata.obs['annotation'] = new_annotations

In [36]:
adata.obs['pheno_annotate'] = new_pheno

In [38]:
adata.obs['annotation'].value_counts()

annotation
all_Malignant                          63861
all_CAF                                41077
all_Ductal                             18911
all_Vascular                           14649
all_Pericyte                           11474
all_myCAF                               9878
Mac                                     9452
all_Vascular smooth muscle              4667
all_Acinar                              4197
all_Ductal (atypical)                   3424
all_Beta                                2903
all_ADM                                 2677
all_Alpha                               2234
CD4-T-naive                             1770
all_Schwann                             1714
all_Hormone-negative neuroendocrine     1599
all_Lymphatic                           1552
all_Gamma                               1451
CD8-T_KLRG1pos-effector                 1404
CD8-TRM                                 1285
T                                       1124
CD8-T                                    716

In [40]:
adata.obs['pheno_annotate'].value_counts()

pheno_annotate
all_Malignant                   63861
all_CAF                         41077
all_Ductal                      18911
all_Vascular                    14649
all_Pericyte                    11474
                                ...  
BP_9                               34
BP_10                              31
all_Intra-pancreatic neurons       25
ILC_3                              25
ILC_4                              17
Name: count, Length: 77, dtype: int64

In [41]:
adata[adata.obs['pheno_annotate'].isna()]

View of AnnData object with n_obs × n_vars = 0 × 22164
    obs: 'sampleid', 'n_genes', 'n_counts', 'scrublet_scores', 'pid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'n_counts_sat', 'n_genes_sat', 'percent_mito', 'leiden', 'batch', 'Moffitt_basal', 'Moffitt_classical', 'Bailey_squamous', 'Bailey_progenitor', 'Collison_QM', 'Collison_classical', 'MALIGNANT CELLS', 'ACINAR', 'Alpha', 'Beta', 'Delta', 'Gamma', 'Episilon', 'ENDOCRINE', 'ENDOTHELIAL', 'Pan_Immune', 'AntigenPresentingCells', 'Monocytes_1', 'Monocytes_2', 'Macrophage', 'cDC1', 'cDC2', 'DC_activated', 'pDC', 'Mast', 'Eosinophils', 'Neutrophils', 'M0', 'M1', 'M2', 'Mast_Resting', 'Mast_activated', 'CD8_Tcells', 'CD4_Tcells', 'NK', 'CD8_gammadelta', 'CD8_exhausted', 'CD4_naive', 'CD4_memory_resting', 'CD4_memory_activated', 'CD4_follicular_helper', 'CD4_regulatory', 'NK_resting', 'NK_activated', 'B_cell', 'Plasma', 'Bcell_naive', 'Bcell_memory', 'IMMUNE', 'PanCAF', 'iCAF', 'myCAF', 'a

In [42]:
print('mapped annotations:', set(adata.obs['annotation']))

mapped annotations: {'gran', 'cDC2', 'all_Acinar (REG+)', 'B_proliferating', 'CD4-T-naive', 'all_Ductal', 'all_Pericyte', 'all_myCAF', 'T', 'all_Beta', 'Mac', 'CD8-TRM', 'plasma', 'all_Schwann', 'B-T_doublet', 'all_Acinar', 'all_Lymphatic', 'NK', 'all_Intra-pancreatic neurons', 'all_CAF', 'all_Alpha', 'all_Malignant', 'all_Hormone-negative neuroendocrine', 'B-naive', 'c-mono', 'CD8-T', 'CD8-T_KLRG1pos-effector', 'all_Gamma', 'all_Epsilon', 'PV-Mac', 'all_Vascular smooth muscle', 'all_Adipocyte', 'Treg', 'cDC1', 'nc-mono', 'CD56bright-NK', 'p-DC', 'B-memory-DN', 'cDC3', 'all_Vascular', 'B', 'GC-B', 'CD4-TSCM_TFH', 'mast', 'CD56dim-NK', 'T_proliferating', 'all_Ductal (atypical)', 'all_Delta', 'all_ADM'}


In [43]:
print('mapped pheno for annotations:', set(adata.obs['pheno_annotate']))

mapped pheno for annotations: {'M_15', 'ILC_4', 'BP_2', 'all_Beta', 'TNK_14', 'M_21', 'all_Lymphatic', 'TNK_pheno_var_TNK_k10_16', 'TNK_5', 'M_14', 'TNK_16', 'BP_0', 'all_Gamma', 'all_Vascular', 'ILC_1', 'all_ADM', 'M_18', 'TNK_2', 'all_Acinar (REG+)', 'BP_8', 'all_Ductal', 'all_Pericyte', 'TNK_7', 'TNK_15', 'TNK_4', 'TNK_0', 'M_22', 'M_8', 'all_Vascular smooth muscle', 'M_10', 'BP_1', 'M_17', 'TNK_12', 'all_Acinar', 'BP_7', 'M_2', 'M_5', 'ILC_3', 'TNK_11', 'M_13', 'all_myCAF', 'M_6', 'M_4', 'M_0', 'all_Intra-pancreatic neurons', 'all_Alpha', 'M_23', 'BP_3', 'BP_5', 'M_3', 'all_Epsilon', 'BP_9', 'all_Adipocyte', 'BP_6', 'M_1', 'all_Delta', 'M_19', 'ILC_2', 'ILC_0', 'M_12', 'BP_4', 'all_Schwann', 'M_20', 'all_CAF', 'all_Malignant', 'TNK_8', 'all_Hormone-negative neuroendocrine', 'TNK_3', 'M_9', 'M_7', 'M_16', 'TNK_1', 'TNK_13', 'TNK_6', 'M_11', 'all_Ductal (atypical)', 'BP_10'}


In [44]:
annotation_df = adata.obs[['pheno_annotate','annotation']]
annotation_df.columns = ['cluster','annotation']
annotation_df

,cluster,annotation
TTTCATGCATGACTGT-1-1,all_Malignant,all_Malignant
ATGAGTCAGGGTACAC-1-1,all_Pericyte,all_Pericyte
TCCTTTCAGAGATGCC-1-1,all_Malignant,all_Malignant
GCAGCCATCCACTTCG-1-1,all_Pericyte,all_Pericyte
TACGTCCGTCGCTTAA-1-1,all_Malignant,all_Malignant
...,...,...
CTGTAGAAGATCGACG-1-45,all_Ductal,all_Ductal
GTGGAAGTCAGAGTTC-1-45,all_Ductal,all_Ductal
CTAACCCAGCTCGAAG-1-45,all_Ductal,all_Ductal
CTCCTTTCACAATGAA-1-45,all_Ductal,all_Ductal


In [45]:
adata_save_path

'/data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_annotated.h5ad'

In [46]:
adata.write(adata_save_path)
print('saved to:',adata_save_path)

saved to: /data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells_Milo_annotated.h5ad


In [51]:
csv_save_path

'/data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-annotations.csv'

In [52]:
annotation_df.to_csv(csv_save_path)
print('saved to:',csv_save_path)

saved to: /data/peer/wallet/Jupyter/2023_Particle_SASP/data/TIL-X-PAAD-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-annotations.csv
